In [1]:
import pathlib
import mio
import collada as co
import xmltodict
import shapely
from shapely.geometry import Polygon
import pandas as pd
import geopandas as gpd

In [2]:
def dae2poly(model_path, x_shift, y_shift):
    """convert a collida model to shapely polygon"""
    model_path = str(model_path)
    mesh = co.Collada(model_path)
    # make polygonlist
    polys = []
    for geom in mesh.geometries:
        for triset in geom.primitives:
            for tri in triset:
                poly = Polygon(tri.vertices)
                polys.append(poly)
                
    gdf = gpd.GeoDataFrame(geometry=polys)
    gdf['id'] = 0
    gdf = gdf.dissolve('id')
    assert len(gdf), 'expect exactly one poy'
    poly = gdf.geometry[0]
    poly = shapely.affinity.translate(poly, xoff=x_shift, yoff=y_shift)
    return poly

In [3]:
def get_coords_df(kml_path):
    """read kml and return dataframe with coords and model name"""
    with open(kml_path) as fin:
        skml = fin.read()
    dic = xmltodict.parse(skml)
    pms = dic['kml']['Document']['Placemark']
    
    dics = []
    for pm in pms:
        model = pm['Model']
        lon = pm['Model']['Location']['longitude']
        lat = pm['Model']['Location']['latitude']
        model = pm['Model']['Link']['href']
        dic = {
            'lon' : lon, 
            'lat' : lat,
            'model' : model
        }
        dics.append(dic)
    #breakpoint()
    df = pd.DataFrame(dics)
    
    # add swiss coordinates
    coords = list(zip(df.lon, df.lat))
    coords = [mio.wgs_swiss(*ll) for ll in coords]
    coords = list(zip(*coords))
    df['x'] = coords[0]
    df['y'] = coords[1]
    return df

In [12]:
def make_gdf(df, kml_file):
    kml_dir = pathlib.Path(kml_file) / '..'
    assert kml_dir.is_dir(), f"kml_dir '{kml_dir}' not found"
    geometry = []
    models = []
    for ind, row in df.iterrows():
        model = row['model']
        x, y = row['x'], row['y']
        poly = dae2poly(kml_dir / model, x, y)
        geometry.append(poly)
        models.append(model)
    gdf = gpd.GeoDataFrame(geometry=geometry)
    gdf['model'] = models
    return gdf

In [13]:
def convert(kml_path, result_dir):
    stem = pathlib.Path(kml_path).stem
    result_dir = pathlib.Path(result_dir)
    assert result_dir.is_dir(), f"{result_dir} not found"
    df = get_coords_df(kml_path)
    gdf = make_gdf(df, kml_path)
    mio.write_tab(gdf, result_dir / (stem + '.tab') )

In [16]:
# find kmls
all_kmls = []
dirs = list(pathlib.Path('KML').glob('*'))
for dir in dirs:
    kmls = list(pathlib.Path(dir).glob('*.kml'))
    all_kmls.extend(kmls)
all_kmls

[WindowsPath('KML/Sample_swissBUILDINGS3D20_WGS84_1166-42/swissBUILDINGS3D20_1166-42.kml'),
 WindowsPath('KML/Sample_swissBUILDINGS3D20_WGS84_1166-44/swissBUILDINGS3D20_1166-44.kml')]

In [17]:
for kml in all_kmls:
    print('converting', kml)
    convert(kml, 'result/')

convering KML\Sample_swissBUILDINGS3D20_WGS84_1166-42\swissBUILDINGS3D20_1166-42.kml


C:\tools\anaconda3\lib\site-packages\collada\util.py:70: RuntimeWarning: invalid value encountered in true_divide
  return vec / numpy.sqrt(numpy.vdot(vec, vec))


1503 row(s) written to mapinfo file.
convering KML\Sample_swissBUILDINGS3D20_WGS84_1166-44\swissBUILDINGS3D20_1166-44.kml
112 row(s) written to mapinfo file.
